In [ ]:
import numpy as np
import cupy as cp
import skimage.io
import skimage.color
import matplotlib.pyplot as plt
from skimage.transform import downscale_local_mean
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from numba import njit
import time


: 

In [ ]:
@njit
def mirror_cpu(A, f):
    n, m = A.shape
    nlin = n + 2*f
    ncol = m + 2*f
    B = np.zeros((nlin, ncol), dtype=A.dtype)

    B[f:n+f, f:m+f] = A
    B[0:f, f:m+f] = A[0:f, :][::-1, :]
    B[n+f:, f:m+f] = A[n-f:, :][::-1, :]
    B[f:n+f, 0:f] = A[:, 0:f][:, ::-1]
    B[f:n+f, m+f:] = A[:, m-f:][:, ::-1]
    B[0:f, 0:f] = A[0:f, 0:f][::-1, ::-1]
    B[0:f, m+f:] = A[0:f, m-f:][::-1, ::-1]
    B[n+f:, 0:f] = A[n-f:, 0:f][::-1, ::-1]
    B[n+f:, m+f:] = A[n-f:, m-f:][::-1, ::-1]

    return B


In [ ]:
@njit
def NLM_fast_cpu(img, h, f, t):
    m, n = img.shape
    filtrada = np.zeros((m, n), dtype=img.dtype)
    img_n = mirror_cpu(img, f)
    for i in range(m):
        for j in range(n):
            im = i + f
            jn = j + f
            W1 = img_n[im-f:im+f+1, jn-f:jn+f+1]
            rmin = max(im-t, f)
            rmax = min(im+t, m+f-1)
            smin = max(jn-t, f)
            smax = min(jn+t, n+f-1)
            NL = 0.0
            Z = 0.0
            for r in range(rmin, rmax+1):
                for s in range(smin, smax+1):
                    W2 = img_n[r-f:r+f+1, s-f:s+f+1]
                    d2 = np.sum((W1 - W2)**2)
                    sij = np.exp(-d2/(h**2))
                    Z += sij
                    NL += sij * img_n[r, s]
            filtrada[i, j] = NL / Z
    return filtrada


In [ ]:
def mirror_gpu(A, f):
    n, m = A.shape
    B = cp.zeros((n + 2*f, m + 2*f), dtype=A.dtype)
    B[f:n+f, f:m+f] = A
    B[0:f, f:m+f] = A[0:f, :][::-1, :]
    B[n+f:, f:m+f] = A[n-f:, :][::-1, :]
    B[f:n+f, 0:f] = A[:, 0:f][:, ::-1]
    B[f:n+f, m+f:] = A[:, m-f:][:, ::-1]
    B[0:f, 0:f] = A[0:f, 0:f][::-1, ::-1]
    B[0:f, m+f:] = A[0:f, m-f:][::-1, ::-1]
    B[n+f:, 0:f] = A[n-f:, 0:f][::-1, ::-1]
    B[n+f:, m+f:] = A[n-f:, m-f:][::-1, ::-1]
    return B


In [ ]:
nlm_kernel_global_code = r'''
extern "C" __global__
void nlm_kernel_global(
    const float* img_n, float* output,
    int m, int n, int f, int t, float h, int padded_width
) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i >= m || j >= n)
        return;

    int im = i + f;
    int jm = j + f;

    float NL = 0.0f;
    float Z = 0.0f;

    for (int r = im - t; r <= im + t; ++r) {
        for (int s = jm - t; s <= jm + t; ++s) {
            float d2 = 0.0f;
            for (int u = -f; u <= f; ++u) {
                for (int v = -f; v <= f; ++v) {
                    int x1 = im + u;
                    int y1 = jm + v;
                    int x2 = r + u;
                    int y2 = s + v;

                    float val1 = img_n[x1 * padded_width + y1];
                    float val2 = img_n[x2 * padded_width + y2];
                    float diff = val1 - val2;
                    d2 += diff * diff;
                }
            }
            float weight = __expf(-d2 / (h * h));
            Z += weight;
            NL += weight * img_n[r * padded_width + s];
        }
    }

    output[i * n + j] = NL / Z;
}
'''



In [ ]:
def NLM_fast_cuda_global(img, h, f, t):
    img = img.astype(cp.float32)
    m, n = img.shape
    padded = mirror_gpu(img, f)

    module = cp.RawModule(code=nlm_kernel_global_code, options=('-std=c++11',))
    kernel = module.get_function("nlm_kernel_global")

    output = cp.zeros((m, n), dtype=cp.float32)
    threads_per_block = (16, 16)
    grid = ((n + 15) // 16, (m + 15) // 16)

    kernel(
        grid, threads_per_block,
        (
            padded.ravel(), output.ravel(),
            cp.int32(m), cp.int32(n), cp.int32(f), cp.int32(t),
            cp.float32(h), cp.int32(padded.shape[1])
        )
    )
    return output

def add_noise_gaussian(img):
    m, n = img.shape
    sigma = 10
    noise = np.random.normal(0, sigma, (m, n)).astype(np.float32)
    noised = np.clip(img + noise, 0, 255)
    return noised

def add_poisson_noise(img):    
    poisson_img = np.random.poisson(img)    
    return poisson_img

def add_poisson_gaussian_noise(image, gaussian_sigma=25):
    # Apply Poisson noise
    poisson_image = np.random.poisson(image)

    # Apply Gaussian noise
    gaussian_noise = np.random.normal(loc=0.0, scale=gaussian_sigma, size=image.shape)
    noisy_image = poisson_image + gaussian_noise 

    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    
    return noisy_image

def downscale(img):
    img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
    return img_downscale

def anscombe_transform(img):
    return 2.0 * np.sqrt(img.astype(np.float32) + 3.0 / 8.0)

def inverse_anscombe(y):
    # Inversa exata de Makitalo & Foi (IEEE T-IP 2011)
    return ((y / 2.0)**2) - 3.0 / 8.0

def compute_adaptive_q(sigma_est, anscombe=False):
    q_nlm = 0.8 + 0.5 * np.tanh(0.3 * (sigma_est - 1))
    q_geo = 1.0 + 0.7 * np.tanh(0.25 * (sigma_est - 1.5))

    if anscombe:
        q_nlm = int(np.clip(q_nlm, 0.7, 2.2) * 10)
        q_geo = int(np.clip(q_geo, 0.9, 2.7) * 10)
    else:
        q_nlm = int(np.clip(q_nlm, 0.7, 2.2) * 100)
        q_geo = int(np.clip(q_geo, 0.9, 2.7) * 100)

    print(f'q_nlm: {q_nlm}, g_geo: {q_geo}')
    return q_nlm, q_geo

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def select_best_h_using_adaptive_q(image_downscale, image_gpu, q_nlm_candidates, f, t, alpha=0.5, anscombe=False):
    melhor_score = -float('inf')
    melhor_q_nlm = None
    melhor_resultado = None
    melhor_psnr = None
    melhor_ssim = None

    for h_nlm in q_nlm_candidates:
        #print(f'h_nlm: {h_nlm}')

        # Aplica NLM
        result_gpu = NLM_fast_cuda_global(image_gpu, h_nlm, f, t)
        cp.cuda.Stream.null.synchronize()

        # Referência: imagem original
        img_ref = np.clip(image_downscale, 0, 255).astype(np.uint8)

        if anscombe:
            # Inversa de Anscombe
            result_processed = np.maximum(inverse_anscombe(cp.asnumpy(result_gpu)), 0)
        else:
            result_processed = cp.asnumpy(result_gpu)

        result_uint8 = np.clip(result_processed, 0, 255).astype(np.uint8)

        # Métricas
        psnr = peak_signal_noise_ratio(img_ref, result_uint8, data_range=255)
        ssim = structural_similarity(img_ref, result_uint8, data_range=255)

        # Score combinado
        score = alpha * psnr + (1 - alpha) * (ssim * 100)
        print(f"h = {h_nlm:.2f} | PSNR = {psnr:.2f} | SSIM = {ssim:.4f} | Score = {score:.2f}")

        if score > melhor_score:
            melhor_score = score
            melhor_q_nlm = h_nlm
            melhor_resultado = result_processed
            melhor_psnr = psnr
            melhor_ssim = ssim

    print(f"\n[SELECIONADO] H = {melhor_q_nlm:.2f} | PSNR = {melhor_psnr:.2f} | SSIM = {melhor_ssim:.4f} | SCORE = {melhor_score:.2f}")

    return melhor_resultado, melhor_q_nlm, melhor_psnr, melhor_ssim, melhor_score


### Testando

In [ ]:
# ------------------------------- Execução principal -------------------------------

# Carregamento da imagem
img_path = '../images/0.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img

# Downscale + ruído
img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
m, n = img_downscale.shape
sigma = 10
ruido = np.random.normal(0, sigma, (m, n)).astype(np.float32)
ruidosa = np.clip(img_downscale + ruido, 0, 255)

# Preparo para CPU e GPU
img_cpu = ruidosa.astype(np.float32)
img_gpu = cp.array(img_cpu)

# Parâmetros do filtro
h = 110
f = 4
t = 7

# ------------------------------- CPU -------------------------------
start_cpu = time.time()
result_cpu = NLM_fast_cpu(img_cpu, h, f, t)
print("Tempo CPU:", time.time() - start_cpu)

# ------------------------------- GPU -------------------------------
start_gpu = time.time()
result_gpu = NLM_fast_cuda_global(img_gpu, h, f, t)
cp.cuda.Stream.null.synchronize()
print("Tempo GPU:", time.time() - start_gpu)

# Comparação
result_gpu_cpu = cp.asnumpy(result_gpu)
diff = np.abs(result_cpu - result_gpu_cpu)
print(f'Diferença média: {diff.mean():.4f}')
print(f'Diferença máxima: {diff.max():.4f}')

# Métricas PSNR e SSIM em relação à imagem original
img_ref = np.clip(img_downscale, 0, 255).astype(np.uint8)
result_cpu_uint8 = np.clip(result_cpu, 0, 255).astype(np.uint8)
result_gpu_uint8 = np.clip(result_gpu_cpu, 0, 255).astype(np.uint8)

psnr_cpu = peak_signal_noise_ratio(img_ref, result_cpu_uint8, data_range=255)
psnr_gpu = peak_signal_noise_ratio(img_ref, result_gpu_uint8, data_range=255)
ssim_cpu = structural_similarity(img_ref, result_cpu_uint8, data_range=255)
ssim_gpu = structural_similarity(img_ref, result_gpu_uint8, data_range=255)

print(f'\n--- Métricas de Qualidade ---')
print(f'PSNR (CPU NLM):  {psnr_cpu:.2f} dB')
print(f'PSNR (CUDA NLM): {psnr_gpu:.2f} dB')
print(f'SSIM (CPU NLM):  {ssim_cpu:.4f}')
print(f'SSIM (CUDA NLM): {ssim_gpu:.4f}')

# Visualizações
plt.figure(figsize=(12,4))
plt.subplot(1,3,1); plt.title("Original"); plt.imshow(img_ref, cmap='gray')
plt.subplot(1,3,2); plt.title("CPU NLM"); plt.imshow(result_cpu, cmap='gray')
plt.subplot(1,3,3); plt.title("GPU NLM"); plt.imshow(result_gpu_cpu, cmap='gray')
plt.tight_layout(); plt.show()

# Diferença absoluta entre CPU e GPU
plt.figure(figsize=(5,4))
plt.title("Diferença absoluta CPU vs GPU")
plt.imshow(diff, cmap='hot')
plt.colorbar(label='Erro absoluto')
plt.tight_layout()
plt.show()

In [ ]:
from skimage.io import imsave
imsave("cpu_result.png", np.clip(result_cpu, 0, 255).astype(np.uint8))
imsave("gpu_result.png", np.clip(result_gpu_cpu, 0, 255).astype(np.uint8))
imsave("abs_diff.png", np.clip(diff * 10, 0, 255).astype(np.uint8))

### Noise Gaussian

In [ ]:
from skimage.restoration import estimate_sigma

# ------------------------------- Execução principal -------------------------------

# Carregamento da imagem
img_path = '../images/0.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img

# Downscale + ruído
img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
m, n = img_downscale.shape
sigma = 10
ruido = np.random.normal(0, sigma, (m, n)).astype(np.float32)
ruidosa = np.clip(img_downscale + ruido, 0, 255)

# Preparo para CPU e GPU
img_cpu = ruidosa.astype(np.float32)
estimated_sigma = estimate_sigma(img_cpu)
img_gpu = cp.array(img_cpu)


h_nlm , g_nlm = compute_adaptive_q(estimated_sigma)

q_nlm_candidates = np.array([h_nlm + delta for delta in range(-50, 25, 1)])

print(f'q_nlm_candidates: {q_nlm_candidates}')
# Parâmetros do filtro

In [ ]:
melhor_img, melhor_h, melhor_psnr, melhor_ssim, score = select_best_h_using_adaptive_q(
    image_downscale=img_downscale,
    image_gpu=img_gpu,
    q_nlm_candidates=q_nlm_candidates,
    f=3, t=7,
    alpha=0.5,
    anscombe=False
)

### Noise Poisson

In [ ]:
from skimage.restoration import estimate_sigma

# ------------------------------- Execução principal -------------------------------

# Carregamento da imagem
img_path = '../images/0.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img

# Downscale + ruído
img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
m, n = img_downscale.shape

ruidosa = add_poisson_noise(img_downscale)
print(f'ruidosa: {type(ruidosa)}')

# Preparo para CPU e GPU
img_cpu = ruidosa.astype(np.float32)
print(f'img_cpu: {type(img_cpu)}')
estimated_sigma = estimate_sigma(img_cpu)
print(f'estimated_sigma: {(estimated_sigma)}')
img_gpu = cp.array(img_cpu)
print(f'img_gpu: {type(img_gpu)}')


h_nlm , g_nlm = compute_adaptive_q(estimated_sigma)

q_nlm_candidates_poisson = np.array([h_nlm + delta for delta in range(-30, 30, 1)])

print(f'q_nlm_candidates: {q_nlm_candidates_poisson}')
# Parâmetros do filtro

In [ ]:
melhor_img, melhor_h, melhor_psnr, melhor_ssim, score = select_best_h_using_adaptive_q(
    image_downscale=img_downscale,
    image_gpu=img_gpu,
    q_nlm_candidates=q_nlm_candidates_poisson,
    f=3, t=7,
    alpha=0.5,
    anscombe=False
)

### Poisson com Anscombe

In [ ]:
from skimage.restoration import estimate_sigma

# ------------------------------- Execução principal -------------------------------

# Carregamento da imagem
img_path = '../images/0.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img

# Downscale + ruído
img_downscale_poisson = downscale_local_mean(img, (2, 2)).astype(np.float32)
m, n = img_downscale.shape

ruidosa = add_poisson_noise(img_downscale_poisson)
print(f'ruidosa: {type(ruidosa)}')

ruidosa_anscombe = anscombe_transform(ruidosa)
print(type(ruidosa_anscombe))

# Preparo para CPU e GPU
img_cpu = ruidosa_anscombe.astype(np.float32)
print(f'img_cpu: {type(img_cpu)}')
estimated_sigma = estimate_sigma(img_cpu, channel_axis=None)
print(f'estimated_sigma: {(estimated_sigma)}')
img_gpu = cp.array(img_cpu)
print(f'img_gpu: {type(img_gpu)}')

h_nlm, g_nlm = compute_adaptive_q(estimated_sigma, anscombe=True)

h_nlm, g_nlm = h_nlm, g_nlm

q_nlm_candidates_poisson_gaussian = np.array(
    [h_nlm + delta for delta in range(-30, 30, 1) if (h_nlm + delta) >= 1]
)

print(f'q_nlm_candidates_poisson_gaussian: {img_downscale_poisson, q_nlm_candidates_poisson_gaussian}')
# Parâmetros do filtro



In [ ]:
melhor_img, melhor_h, melhor_psnr, melhor_ssim, score = select_best_h_using_adaptive_q(
    image_downscale=img_downscale,
    image_gpu=img_gpu,
    q_nlm_candidates=q_nlm_candidates_poisson_gaussian,
    f=3, t=7,
    alpha=0.5,
    anscombe=True
)


### Poisson Gaussian Anscombe

In [ ]:
from skimage.restoration import estimate_sigma

# ------------------------------- Execução principal -------------------------------

# Carregamento da imagem
img_path = '../images/4.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img

# Downscale + ruído
img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
m, n = img_downscale.shape

ruidosa = add_poisson_gaussian_noise(img_downscale)
print(f'ruidosa: {type(ruidosa)}')

ruidosa_anscombe = anscombe_transform(ruidosa)
print(type(ruidosa_anscombe))

# Preparo para CPU e GPU
img_cpu = ruidosa_anscombe.astype(np.float32)
print(f'img_cpu: {type(img_cpu)}')
estimated_sigma = estimate_sigma(img_cpu)# channel_axis=None)
print(f'estimated_sigma: {(estimated_sigma)}')
img_gpu = cp.array(img_cpu)
print(f'img_gpu: {type(img_gpu)}')


h_nlm, g_nlm = compute_adaptive_q(estimated_sigma, anscombe=True)

q_nlm_candidates_poisson_gaussian = np.array(
    [h_nlm + delta for delta in range(-30, 30, 1) if (h_nlm + delta) >= 1]
)

print(f'q_nlm_candidates_poisson_gaussian: {q_nlm_candidates_poisson_gaussian}')

In [ ]:
melhor_img, melhor_h, melhor_psnr, melhor_ssim, score = select_best_h_using_adaptive_q(
    image_downscale=img_downscale,
    image_gpu=img_gpu,
    q_nlm_candidates=q_nlm_candidates_poisson_gaussian,
    f=3, t=7,
    alpha=0.5,
    anscombe=True
)


### Com ERRO

In [ ]:
## COM ERRO!!
# Carregamento da imagem
img_path = '../images/1.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img

img_downscale = downscale(img)

img_noise_gaussian = add_noise_gaussian(img_downscale)

img_noise_poisson = add_poisson_noise(img_downscale)
noised_anscombe_poisson = anscombe_transform(img_noise_poisson)

img_noise_poisson_gaussian = add_poisson_gaussian_noise(img_downscale)
noised_anscombe_poisson_gaussian = anscombe_transform(img_noise_poisson_gaussian)

# Gaussian
img_cpu_gaussian = img_noise_gaussian.astype(np.float32)
print(f'img_cpu_gaussian: {type(img_cpu_gaussian)}')

estimated_sigma_gaussian = estimate_sigma(img_cpu_gaussian)# channel_axis=None)
print(f'estimated_sigma_gaussian: {(estimated_sigma_gaussian)}')
img_gpu_gaussian = cp.array(img_cpu_gaussian)
print(f'img_gpu_gaussian: {type(img_gpu_gaussian)}')

h_nlm_gaussian, g_nlm_gaussian = compute_adaptive_q(estimated_sigma_gaussian)

q_nlm_candidates_gaussian = np.array(
    [h_nlm_gaussian + delta for delta in range(-30, 30, 1) if (h_nlm_gaussian + delta) >= 1]
)

#print(f'q_nlm_candidates: {q_nlm_candidates_gaussian}')

h_nlm_final_gaussian =  select_best_h_using_adaptive_q(img_downscale, img_gpu_gaussian, q_nlm_candidates_gaussian, f=4, t=7, alpha=0.5)


# Poisson
img_cpu_poisson = noised_anscombe_poisson.astype(np.float32)
print(f'img_cpu_poisson: {type(img_cpu_poisson)}')

estimated_sigma_poisson = estimate_sigma(img_cpu_poisson, anscombe=True)
print(f'estimated_sigma_poisson: {(estimated_sigma_poisson)}')
img_gpu_poisson = cp.array(img_cpu_poisson)
print(f'img_gpu_poisson: {type(img_gpu_poisson)}')

h_nlm_poisson, g_nlm_poisson = compute_adaptive_q(estimated_sigma_poisson, anscombe=True)

q_nlm_candidates_poisson = np.array(
    [h_nlm_poisson + delta for delta in range(-30, 30, 1) if (h_nlm_poisson + delta) >= 1]
)

#print(f'q_nlm_candidates: {q_nlm_candidates_poisson}')

h_nlm_final_poisson =  select_best_h_using_adaptive_q(img_downscale, img_gpu_poisson, q_nlm_candidates_poisson, f=4, t=7, alpha=0.5, anscombe=True)


# Poisson Gaussian
img_cpu_poisson_gaussian = noised_anscombe_poisson_gaussian.astype(np.float32)
print(f'img_cpu_poisson_gaussian: {type(img_cpu_poisson_gaussian)}')

estimated_sigma_poisson_gaussian = estimate_sigma(img_cpu_poisson_gaussian, anscombe=True)# channel_axis=None)
print(f'estimated_sigma_poisson_gaussian: {(estimated_sigma_poisson_gaussian)}')
img_gpu_poisson_gaussian = cp.array(img_cpu_poisson_gaussian)
print(f'img_gpu_poisson_gaussian: {type(img_gpu_poisson_gaussian)}')

h_nlm_poisson_gaussian, g_nlm_poisson_gaussian = compute_adaptive_q(estimated_sigma_poisson_gaussian, anscombe=True)

q_nlm_candidates_poisson_gaussian = np.array(
    [h_nlm_poisson_gaussian + delta for delta in range(-30, 30, 1) if (h_nlm_poisson_gaussian + delta) >= 1]
)

#print(f'q_nlm_candidates: {q_nlm_candidates_poisson_gaussian}')

melhor_img, melhor_h, melhor_psnr, melhor_ssim = select_best_h_using_adaptive_q(
    image_downscale=img_downscale,
    image_gpu=img_gpu,
    q_nlm_candidates=q_nlm_candidates_poisson_gaussian,
    f=4, t=7,
    alpha=0.5,
    anscombe=True
)




### Pipeline Filtragem NLM

In [ ]:
def pipeline_filtragem_nlm(params):
    """
    Executa o pipeline completo de:
    - aplicação de ruído (gaussian, poisson, poisson_gaussian),
    - transformada de Anscombe (se necessário),
    - estimativa de sigma,
    - escolha adaptativa de h,
    - filtragem com NLM CUDA.

    Parâmetros esperados no dicionário 'params':
    - img_path: str
    - tipo_ruido: 'gaussian', 'poisson' ou 'poisson_gaussian'
    - f: int (tamanho do patch)
    - t: int (janela de busca)
    - alpha: float (peso entre PSNR e SSIM)
    - gaussian_sigma: float (para ruído Poisson-Gaussiano)
    """
    import skimage.io
    import skimage.color
    from skimage.transform import downscale_local_mean
    import numpy as np
    import cupy as cp
    from skimage.restoration import estimate_sigma
    from skimage.metrics import peak_signal_noise_ratio, structural_similarity

    # Carregamento da imagem
    img = skimage.io.imread(params["img_path"])
    img = img[0, :, :] if len(img.shape) > 2 else img
    if len(img.shape) > 2:
        img = skimage.color.rgb2gray(img)
        img = 255 * img

    if downscale:
        img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
    else:
        img_downscale = img.copy()

    tipo_ruido = params["tipo_ruido"]
    f, t = params.get("f", 4), params.get("t", 7)
    alpha = params.get("alpha", 0.5)
    

    # Aplicação do ruído + Anscombe
    if tipo_ruido == "gaussian":
        img_noisy = add_noise_gaussian(img_downscale)
        img_cpu = img_noisy.astype(np.float32)
        anscombe = False
    elif tipo_ruido == "poisson":
        img_noisy = add_poisson_noise(img_downscale)
        img_cpu = anscombe_transform(img_noisy).astype(np.float32)
        anscombe = True
    elif tipo_ruido == "poisson_gaussian":
        sigma = params.get("gaussian_sigma", 25)
        img_noisy = add_poisson_gaussian_noise(img_downscale, gaussian_sigma=sigma)
        img_cpu = anscombe_transform(img_noisy).astype(np.float32)
        anscombe = True
    else:
        raise ValueError("Tipo de ruído inválido. Use 'gaussian', 'poisson' ou 'poisson_gaussian'.")

    # Estimativa de sigma
    estimated_sigma = estimate_sigma(img_cpu)
    h_nlm, _ = compute_adaptive_q(estimated_sigma, anscombe=anscombe)

    # Geração dos candidatos a h
    q_nlm_candidates = np.array(
        [h_nlm + delta for delta in range(-50, 50, 1) if (h_nlm + delta) >= 1]
    )

    img_gpu = cp.array(img_cpu)

    # Seleção do melhor h
    melhor_img, melhor_h, melhor_psnr, melhor_ssim, score = select_best_h_using_adaptive_q(
        image_downscale=img_downscale,
        image_gpu=img_gpu,
        q_nlm_candidates=q_nlm_candidates,
        f=f, t=t,
        alpha=alpha,
        anscombe=anscombe
    )

    return {
        "imagem_filtrada": melhor_img,
        "melhor_h": melhor_h,
        "psnr": melhor_psnr,
        "ssim": melhor_ssim,
        "score": score,
        "imagem_ruidosa": img_noisy,
        "imagem_original": img_downscale
    }


In [ ]:
resultados = pipeline_filtragem_nlm({
    "img_path": "../images/1.gif",
    "tipo_ruido": "gaussian",
    "f": 4,
    "t": 7,
    "alpha": 0.5,
    "gaussian_sigma": 20,
    "downscale":True,
})

# Acessar resultados:
img_filtrada = resultados["imagem_filtrada"]
print(f"PSNR: {resultados['psnr']:.10f}, SSIM: {resultados['ssim']:.10f}, H: {resultados['melhor_h']}, SCORE: {resultados['score']:.2f}")


### Paralel

In [ ]:
import sklearn.neighbors as sknn
import networkx as nx

def process_pixel(i, j, img_n, f, t, h, nn, m, n):
    im = i + f
    jn = j + f
    patch_central = img_n[im - f:im + f + 1, jn - f:jn + f + 1]
    central = patch_central.ravel()

    rmin = max(im - t, f)
    rmax = min(im + t, m + f)
    smin = max(jn - t, f)
    smax = min(jn + t, n + f)

    n_patches = (rmax - rmin) * (smax - smin)
    patch_size = (2 * f + 1) ** 2

    dataset = np.empty((n_patches, patch_size), dtype=np.float32)
    pixels_busca = np.empty(n_patches, dtype=np.float32)

    source = -1
    k = 0
    for r in range(rmin, rmax):
        for s in range(smin, smax):
            W = img_n[r - f:r + f + 1, s - f:s + f + 1]
            neighbor = W.ravel()
            dataset[k, :] = neighbor
            pixels_busca[k] = img_n[r, s]
            if np.array_equal(central, neighbor):
                source = k
            k += 1

    if source == -1:
        source = 0  # fallback para evitar erro

    knnGraph = sknn.kneighbors_graph(dataset, n_neighbors=nn, mode='distance')
    G = nx.from_scipy_sparse_array(knnGraph)
    length, _ = nx.single_source_dijkstra(G, source)

    points = list(length.keys())
    distancias = np.array(list(length.values()), dtype=np.float32)

    similaridades = np.exp(-distancias ** 2 / (h ** 2))
    pixels = pixels_busca[points]

    NL = np.sum(similaridades * pixels)
    Z = np.sum(similaridades)
    return NL / Z if Z > 0 else img_n[im, jn]


from joblib import Parallel, delayed

def Parallel_GEONLM(img_n, f, t, h, nn):
    print(f'img_n.shape: {img_n.shape}')
    m = img_n.shape[0] - 2 * f
    n = img_n.shape[1] - 2 * f
    print(f'M: {m}, N: {n}')

    filtrada = Parallel(n_jobs=-1)(
        delayed(process_pixel)(i, j, img_n, f, t, h, nn, m, n)
        for i in range(m)
        for j in range(n)
    )

    filtrada_geo = np.array(filtrada).reshape((m, n))
    return filtrada_geo


In [ ]:
import numpy as np
import skimage.io
import skimage.color
from skimage.restoration import estimate_sigma
from skimage.transform import downscale_local_mean
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

# Assuma que essas funções estão definidas:
# - downscale
# - mirror_cpu
# - add_noise_gaussian
# - compute_adaptive_q
# - Parallel_GEONLM (otimizada)
# - process_pixel (otimizada)

def run_geonlm_pipeline(img_original, img_noisy, f, t, h_candidates, nn, downscale):

    # Downscale das imagens
    img_downscale_64 = downscale_to_64(img_original)
    img_ruidosa_downscale_64 = downscale_to_64(img_noisy)    
    img_noisy_mirror_64 = mirror_cpu(img_ruidosa_downscale_64, f)

    if downscale:
        img_original_2 = downscale_local_mean(img_original, (2, 2)).astype(np.float32)
        img_noisy_2 = downscale_local_mean(img_noisy, (2, 2)).astype(np.float32)
        img_noisy_mirror = mirror_cpu(img_noisy_2, f)
    else:
        img_original_2 = img_original.copy()
        img_noisy_mirror = mirror_cpu(img_noisy, f)         

    # Verificação das dimensões das imagens
    print(f"Dimensões da imagem original: {img_original.shape}")
    print(f"Dimensões da imagem ruidosa: {img_noisy.shape}")
    print(f"Dimensões da imagem original (downscale 64): {img_downscale_64.shape}")
    print(f"Dimensões da imagem ruidosa (downscale 64): {img_ruidosa_downscale_64.shape}")
    print(f"Dimensões da imagem ruidosa (mirror): {img_noisy_mirror_64.shape}")

    melhor_score = -np.inf
    melhor_img = None
    melhor_h = None
    melhor_psnr = None
    melhor_ssim = None

    for h in h_candidates:
        print(f"\nExecutando GEONLM com h = {h}")
        
        # Versão com paralelismo (ficou mais rápida)
        img_n_geo = np.pad(img_noisy_mirror_64, ((f, f), (f, f)), 'symmetric')

        # Filtragem GEONLM
        img_geo = Parallel_GEONLM(img_n_geo, f=f, t=t, h=h, nn=nn)

        img_geo_no_pad = img_geo[f:-f, f:-f]  # Remove 'f' pixels de cada lado
        img_geo_no_pad = np.clip(img_geo_no_pad, 0, 255).astype(np.uint8)

        # Verificação das dimensões da imagem filtrada
        print(f"Dimensões da imagem filtrada: {img_geo.shape}")

        # Definindo imagem de referência
        img_ref = np.clip(img_downscale_64, 0, 255).astype(np.uint8)

        print(f"Dimensões da imagem Referência: {img_ref.shape}")
        
        # Calculando PSNR e SSIM para a imagem filtrada
        psnr = peak_signal_noise_ratio(img_ref, img_geo_no_pad, data_range=255)
        ssim = structural_similarity(img_ref, img_geo_no_pad, data_range=255)

        # Calculando o score
        score = 0.5 * psnr + 0.5 * (ssim * 100)
        print(f"→ PSNR: {psnr:.2f}, SSIM: {ssim:.4f}, Score: {score:.2f}")

        # Verificando o melhor resultado
        if score > melhor_score:
            melhor_score = score
            melhor_img = img_geo
            melhor_h = h
            melhor_psnr = psnr
            melhor_ssim = ssim

    # Impressão do melhor resultado encontrado
    print(f"\n[FINAL] Melhor h: {melhor_h}, PSNR: {melhor_psnr:.2f}, SSIM: {melhor_ssim:.4f}, SCORE: {melhor_score:.4f} ")

    # Filtragem final com o melhor h
    img_n_geo_final = np.pad(img_noisy_mirror, ((f, f), (f, f)), 'symmetric')

    # Filtragem final com o melhor h
    img_geo_final = Parallel_GEONLM(img_n_geo_final, f=f, t=t, h=melhor_h, nn=nn)

    img_geo_no_pad_final = img_geo_final[f:-f, f:-f]  # Remove 'f' pixels de cada lado
    img_geo_no_pad_final_uint8 = np.clip(img_geo_no_pad_final, 0, 255).astype(np.uint8)

    # Verificar dimensões da imagem final filtrada
    print(f"Dimensões da imagem final filtrada: {img_geo_final.shape}")

    # Calculando PSNR e SSIM para a imagem final filtrada
    img_ref_final = np.clip(img_original_2, 0, 255).astype(np.uint8)
  

    # Verificar dimensões das imagens para comparação
    print(f"Dimensões da imagem de referência final: {img_ref_final.shape}")
    print(f"Dimensões da imagem final filtrada (uint8): {img_geo_no_pad_final_uint8.shape}")

    psnr_final = peak_signal_noise_ratio(img_ref_final, img_geo_no_pad_final_uint8, data_range=255)
    ssim_final = structural_similarity(img_ref_final, img_geo_no_pad_final_uint8, data_range=255)

    score_final = 0.5 * psnr_final + 0.5 * (ssim_final * 100)

    return img_geo_no_pad_final_uint8, melhor_h, melhor_psnr, melhor_ssim, melhor_score, psnr_final, ssim_final, score_final




# ------------------------- FUNÇÕES AUXILIARES -------------------------

def downscale_to_64(img):
    factor_h = img.shape[0] // 64
    factor_w = img.shape[1] // 64
    return downscale_local_mean(img, (factor_h, factor_w)).astype(np.float32)

def crop_center(img, size=(64, 64)):
    h, w = img.shape
    ch, cw = size
    start_h = h // 2 - ch // 2
    start_w = w // 2 - cw // 2
    return img[start_h:start_h+ch, start_w:start_w+cw].astype(np.float32)

# def run_single_pipeline_geonlm(img_base,img_noisy, f, t, h_candidates, nn):  
#     img_downscale_64 = downscale_to_64(img_base)
#     img_ruidosa_downscale_64 = downscale_to_64(img_noisy)
#     img_noisy_mirror = mirror_cpu(img_ruidosa_downscale_64, f)
#     melhor_img, melhor_h, psnr, ssim, score = run_geonlm_pipeline(
#         img_original=img_downscale_64,
#         img_noisy=img_noisy_mirror,
#         f=f, t=t,
#         h_candidates=h_candidates,
#         nn=nn,       
#     )
    
#     return img_base, img_noisy, melhor_img, melhor_h, psnr, ssim, score

In [ ]:
def run_geonlm_pipelineTESTE(img_original, img_noisy, f, t, h_candidates, nn, downscale):

    # Downscale das imagens
    img_downscale_64 = downscale_to_64(img_original)
    img_ruidosa_downscale_64 = downscale_to_64(img_noisy)    
    img_noisy_mirror_64 = mirror_cpu(img_ruidosa_downscale_64, f)

    if downscale:
        img_original_2 = downscale_local_mean(img_original, (2, 2)).astype(np.float32)
        img_noisy_2 = downscale_local_mean(img_noisy, (2, 2)).astype(np.float32)
        img_noisy_mirror = mirror_cpu(img_noisy_2, f)
    else:
        img_original_2 = img_original.copy()
        img_noisy_mirror = mirror_cpu(img_noisy, f)         

    # Verificação das dimensões das imagens
    print(f"Dimensões da imagem original: {img_original.shape}")
    print(f"Dimensões da imagem ruidosa: {img_noisy.shape}")
    print(f"Dimensões da imagem original (downscale 64): {img_downscale_64.shape}")
    print(f"Dimensões da imagem ruidosa (downscale 64): {img_ruidosa_downscale_64.shape}")
    print(f"Dimensões da imagem ruidosa (mirror): {img_noisy_mirror_64.shape}")

    melhor_score = -np.inf
    melhor_img = None
    melhor_h = None
    melhor_psnr = None
    melhor_ssim = None

    for h in h_candidates:
        print(f"\nExecutando GEONLM com h = {h}")
        
        # Versão com paralelismo (ficou mais rápida)
        img_n_geo = np.pad(img_noisy_mirror_64, ((f, f), (f, f)), 'symmetric')

        # Filtragem GEONLM
        img_geo = Parallel_GEONLM(img_n_geo, f=f, t=t, h=h, nn=nn)

        # Verificação das dimensões da imagem filtrada
        print(f"Dimensões da imagem filtrada: {img_geo.shape}")

        # Remover o padding da imagem filtrada
        img_geo_no_pad = img_geo[f:-f, f:-f]  # Remover 'f' pixels de cada lado
        print(f"Dimensões da imagem filtrada sem padding: {img_geo_no_pad.shape}")

        # Definindo imagem de referência
        img_ref = np.clip(img_downscale_64, 0, 255).astype(np.uint8)
        
        # Calculando PSNR e SSIM para a imagem filtrada (sem padding)
        psnr = peak_signal_noise_ratio(img_ref, img_geo_no_pad, data_range=255)
        ssim = structural_similarity(img_ref, img_geo_no_pad, data_range=255)

        # Calculando o score
        score = 0.5 * psnr + 0.5 * (ssim * 100)
        print(f"→ PSNR: {psnr:.2f}, SSIM: {ssim:.4f}, Score: {score:.2f}")

        # Verificando o melhor resultado
        if score > melhor_score:
            melhor_score = score
            melhor_img = img_geo_no_pad
            melhor_h = h
            melhor_psnr = psnr
            melhor_ssim = ssim

    # Impressão do melhor resultado encontrado
    print(f"\n[FINAL] Melhor h: {melhor_h}, PSNR: {melhor_psnr:.2f}, SSIM: {melhor_ssim:.4f}, SCORE: {melhor_score:.4f} ")

    # Filtragem final com o melhor h
    img_n_geo_final = np.pad(img_noisy_mirror, ((f, f), (f, f)), 'symmetric')

    # Filtragem final com o melhor h
    img_geo_final = Parallel_GEONLM(img_n_geo_final, f=f, t=t, h=melhor_h, nn=nn)

    # Remover o padding da imagem final filtrada
    img_geo_final_no_pad = img_geo_final[f:-f, f:-f]

    # Verificar dimensões da imagem final filtrada sem padding
    print(f"Dimensões da imagem final filtrada sem padding: {img_geo_final_no_pad.shape}")

    # Calculando PSNR e SSIM para a imagem final filtrada
    img_ref_final = np.clip(img_original_2, 0, 255).astype(np.uint8)
    img_geo_uint8_final = np.clip(img_geo_final_no_pad, 0, 255).astype(np.uint8)

    psnr_final = peak_signal_noise_ratio(img_ref_final, img_geo_uint8_final, data_range=255)
    ssim_final = structural_similarity(img_ref_final, img_geo_uint8_final, data_range=255)

    score_final = 0.5 * psnr_final + 0.5 * (ssim_final * 100)

    return img_geo_final_no_pad, melhor_h, melhor_psnr, melhor_ssim, melhor_score, psnr_final, ssim_final, score_final



# ------------------------- EXEMPLO DE USO COM 3 ABORDAGENS -------------------------

# Parâmetros
f = 4
t = 7
nn = 10

# Carregamento
img_path = '../images/0.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img
img = img.astype(np.float32)

# Abordagens
originals = {
     "Downscale x2": downscale_local_mean(img, (2, 2)).astype(np.float32),
    "Downscale 64x64": downscale_to_64(img),
    "Crop Central 64x64": crop_center(img, (64, 64))  # Apenas cortar a imagem central para 64x64
}

# Inicializando um dicionário para armazenar os resultados
results = {}

h_candidates = [100,101,102]
# Processando cada variante de imagem
for label, img_variant in originals.items():
    print(f"\n--- Executando {label} ---")
    img_geo_final_no_pad, melhor_h, melhor_psnr, melhor_ssim, melhor_score, psnr_final, ssim_final, score_final = run_geonlm_pipelineTESTE(img_variant,imagem_ruidosa, f, t, h_candidates, nn, downscale=False)
    results[label] = (img_geo_final_no_pad, melhor_h, melhor_psnr, melhor_ssim, melhor_score, psnr_final, ssim_final, score_final)

# ------------------------- VISUALIZAÇÃO -------------------------

import matplotlib.pyplot as plt

# Exibindo os resultados de cada abordagem
for label, (original, noisy, filtered, h, psnr, ssim) in results.items():
    plt.figure(figsize=(12, 3))
    plt.suptitle(f"{label} | h={h} | PSNR={psnr:.2f} | SSIM={ssim:.4f}")
    
    # Exibindo as imagens de forma organizada
    plt.subplot(1, 3, 1); plt.title("Original"); plt.imshow(original.astype(np.uint8), cmap='gray'); plt.axis('off')
    plt.subplot(1, 3, 2); plt.title("Ruidosa"); plt.imshow(noisy.astype(np.uint8), cmap='gray'); plt.axis('off')
    plt.subplot(1, 3, 3); plt.title("GEONLM"); plt.imshow(filtered.astype(np.uint8), cmap='gray'); plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
resultados = pipeline_filtragem_nlm({
    "img_path": "../images/0.gif",
    "tipo_ruido": "gaussian",
    "f": 4,
    "t": 7,
    "alpha": 0.5,
    "gaussian_sigma": 20
})

# Acessar resultados:
img_filtrada = resultados["imagem_filtrada"]
print(f"PSNR: {resultados['psnr']:.10f}, SSIM: {resultados['ssim']:.10f}, H: {resultados['melhor_h']}, SCORE: {resultados['score']:.2f}")


In [ ]:
parametros_comuns = {
    "img_path": "../images/0.gif",
    "tipo_ruido": "gaussian",
    "f": 4,
    "t": 7,
    "alpha": 0.5,
    "gaussian_sigma": 20,
    "downscale": False,
}

resultados_nlm = pipeline_filtragem_nlm(parametros_comuns)

In [ ]:
f = parametros_comuns['f']
t = parametros_comuns['t']
nn = 10

imagem_original = resultados_nlm['imagem_original']
imagem_ruidosa = resultados_nlm['imagem_ruidosa']
melhor_h =  resultados_nlm['melhor_h']
#img_downscale_64 = downscale_to_64(imagem_original)
#img_ruidosa_downscale_64 = downscale_to_64(imagem_ruidosa)

h_candidates = np.array([melhor_h + delta for delta in range(-2, 2, 1)])

resultados_teste_geonlm = run_geonlm_pipeline(imagem_original,imagem_ruidosa, f, t, h_candidates, nn, downscale=False)





In [ ]:

f = 4
t= 7
imagem_original = resultados['imagem_original']
imagem_ruidosa = resultados['imagem_ruidosa']
melhor_h =  resultados['melhor_h']
downscale = True
nn=10
#img_downscale_64 = downscale_to_64(imagem_original)
#img_ruidosa_downscale_64 = downscale_to_64(imagem_ruidosa)

h_candidates = np.array([melhor_h + delta for delta in range(-30, 30, 3)])

resultados_teste_geonlm = run_geonlm_pipeline(imagem_original,imagem_ruidosa, f, t, h_candidates, nn, downscale)



### Como estava

In [ ]:
# def run_single_pipeline(img_base, f, t, nn):
#     img_noisy = add_noise_gaussian(img_base)
#     sigma = estimate_sigma(img_noisy)
#     h_nlm, _ = compute_adaptive_q(sigma)
#     img_noisy_mirror = mirror_cpu(img_noisy, f)

#     melhor_img, melhor_h, psnr, ssim, score= run_geonlm_pipeline(
#         img_original=img_base,
#         img_noisy=img_noisy_mirror,
#         f=f, t=t,
#         h_base=h_nlm,
#         nn=nn
#     )
#     return img_base, img_noisy, melhor_img, melhor_h, psnr, ssim, score


def run_geonlm_pipeline2(img_original, img_noisy, f, t, nn=10, h_multipliers=[0.85, 0.90,0.95,1, 1.05])#, 1.10, 1.15, 1.2, 1.23, 1.25,1.35]):

    img_noisy_mirror = mirror_cpu(img_noisy, f)
    h_base, _ = compute_adaptive_q(sigma)

    # Versão com paralelismo (ficou mais rápida)
    img_n_geo = np.pad(img_noisy_mirror, ((f, f), (f, f)), 'symmetric')

    melhor_score = -np.inf
    melhor_img = None
    melhor_h = None
    melhor_psnr = None
    melhor_ssim = None

    for mult in h_multipliers:
        h_geo = (h_base)  * mult
        print(f"\nExecutando GEONLM com h = {h_geo:.2f} (base {h_base} * {mult})")

        img_geo = Parallel_GEONLM(img_n_geo, f=f, t=t, h=h_geo, nn=nn)

        img_geo_no_pad = img_geo[f:-f, f:-f]  # Remove 'f' pixels de cada lado
        img_geo_no_pad = np.clip(img_geo_no_pad, 0, 255).astype(np.uint8)

        img_ref = np.clip(img_original, 0, 255).astype(np.uint8)
        #img_geo_uint8 = np.clip(img_geo, 0, 255).astype(np.uint8)

        psnr = peak_signal_noise_ratio(img_ref, img_geo_no_pad, data_range=255)
        ssim = structural_similarity(img_ref, img_geo_no_pad, data_range=255)

        score = 0.5 * psnr + 0.5 * (ssim * 100)
        print(f"→ PSNR: {psnr:.2f}, SSIM: {ssim:.4f}, Score: {score:.2f}")

        if score > melhor_score:
            melhor_score = score
            melhor_img = img_geo
            melhor_h = h_geo
            melhor_psnr = psnr
            melhor_ssim = ssim

    print(f"\n[FINAL] Melhor h: {melhor_h}, PSNR: {melhor_psnr:.2f}, SSIM: {melhor_ssim:.4f}, SCORE: {melhor_score:.4f} ")
    return melhor_img, melhor_h, melhor_psnr, melhor_ssim, melhor_score

In [ ]:
# ------------------------- EXEMPLO DE USO COM 3 ABORDAGENS -------------------------

# Parâmetros
f = 4
t = 7
nn = 10

# Carregamento
img_path = '../images/4.gif'
img = skimage.io.imread(img_path)
img = img[0, :, :] if len(img.shape) > 2 else img
if len(img.shape) > 2:
    img = skimage.color.rgb2gray(img)
    img = 255 * img
img = img.astype(np.float32)

# ------------------------- DOWNCALE PARA 64x64 -------------------------
# Redimensionando a imagem para 64x64
img_downscaled = downscale_to_64(img)
imagem_ruidosa_downscaled = downscale_to_64(img)

# ------------------------- EXECUÇÃO DO PIPELINE -------------------------
# Chamando a função que executa o pipeline de filtragem
print("\n--- Executando Pipeline com Downscale 64x64 ---")
# Executar o pipeline com a imagem ruidosa
melhor_img, melhor_h, melhor_psnr, melhor_ssim, melhor_score = run_geonlm_pipeline2(img_downscaled, imagem_ruidosa_downscaled, f, t, nn)

# Exibir o resultado
print(f"Melhor valor de h: {melhor_h}")
print(f"PSNR: {melhor_psnr:.2f}, SSIM: {melhor_ssim:.4f}, Score: {melhor_score:.2f}")

# ------------------------- VISUALIZAÇÃO -------------------------

import matplotlib.pyplot as plt

# Exibindo os resultados
plt.figure(figsize=(12, 3))
# plt.suptitle(f"Downscale 64x64 | h={h_val} | PSNR={psnr_val:.2f} | SSIM={ssim_val:.4f}")

# Exibindo as imagens de forma organizada
plt.subplot(1, 3, 1); plt.title("Original"); plt.imshow(img_downscaled.astype(np.uint8), cmap='gray'); plt.axis('off')
plt.subplot(1, 3, 2); plt.title("Ruidosa"); plt.imshow(imagem_ruidosa_downscaled.astype(np.uint8), cmap='gray'); plt.axis('off')
plt.subplot(1, 3, 3); plt.title("GEONLM"); plt.imshow(melhor_img.astype(np.uint8), cmap='gray'); plt.axis('off')

plt.tight_layout()
plt.show()